In [1]:
import cv2
import numpy as np
import os

image_x, image_y = 50, 50

cap = cv2.VideoCapture(0)
fbag = cv2.createBackgroundSubtractorMOG2()

def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

def main(g_id):
    total_pics = 100
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    create_folder("C:/Users/Pavani/Desktop/gestures/" + str(g_id))
    pic_no = 0
    flag_start_capturing = False
    frames = 0

    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(frame, frame, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)

        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 10000 and frames > 50:
                x1, y1, w1, h1 = cv2.boundingRect(contour)
                pic_no += 1
                save_img = thresh[y1:y1 + h1, x1:x1 + w1]
                if w1 > h1:
                    save_img = cv2.copyMakeBorder(save_img, int((w1 - h1) / 2), int((w1 - h1) / 2), 0, 0,
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                elif h1 > w1:
                    save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1 - w1) / 2), int((h1 - w1) / 2),
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                save_img = cv2.resize(save_img, (image_x, image_y))
                cv2.putText(frame, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
                cv2.imwrite("C:/Users/Pavani/Desktop/gestures/" + str(g_id) + "/" + str(pic_no) + ".jpg", save_img)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, str(pic_no), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
        cv2.imshow("Capturing gesture", frame)
        cv2.imshow("thresh", thresh)
        keypress = cv2.waitKey(1)
        if keypress == ord('c'):
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
                frames = 0
        if flag_start_capturing == True:
            frames += 1
        if pic_no == total_pics:
            break


g_id = input("Enter gesture number: ")
main(g_id)

Enter gesture number: 1


In [21]:

import imageio

import numpy as np
import pandas as pd
import os
root = 'C:/Users/Pavani/Desktop/gestures/' 

# go through each directory in the root folder given above
for directory, subdirectories, files in os.walk(root):
# go through each file in that directory
    for file in files:
    # read the image file and extract its pixels
        print(file)
        im = imageio.imread(os.path.join(directory,file))
        value = im.flatten()
        value = np.hstack((directory[11:],value))
        df = pd.DataFrame(value).T
        df = df.sample(frac=1) # shuffle the dataset
        with open('train_foo.csv', 'a') as dataset:
            df.to_csv(dataset, header=False, index=False)


1.jpg
10.jpg
100.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
5.jpg
50.jpg
51.jpg
52.jpg
53.jpg
54.jpg
55.jpg
56.jpg
57.jpg
58.jpg
59.jpg
6.jpg
60.jpg
61.jpg
62.jpg
63.jpg
64.jpg
65.jpg
66.jpg
67.jpg
68.jpg
69.jpg
7.jpg
70.jpg
71.jpg
72.jpg
73.jpg
74.jpg
75.jpg
76.jpg
77.jpg
78.jpg
79.jpg
8.jpg
80.jpg
81.jpg
82.jpg
83.jpg
84.jpg
85.jpg
86.jpg
87.jpg
88.jpg
89.jpg
9.jpg
90.jpg
91.jpg
92.jpg
93.jpg
94.jpg
95.jpg
96.jpg
97.jpg
98.jpg
99.jpg
1.jpg
10.jpg
100.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.

In [53]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.utils import to_categorical


data = pd.read_csv("train_foo.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 1:2501]
Y = Y[:, 0]


In [54]:
total_images = num_of_classes*100
train_images = int(total_images*0.85)


X_train = X[0:train_images, :]
X_train = X_train / 255.
X_test = X[train_images:total_images, :]
X_test = X_test / 255.


In [55]:
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:train_images, :]
Y_train = Y_train.T
Y_test = Y[train_images:total_images, :]
Y_test = Y_test.T

In [38]:

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))
image_x = 50
image_y = 50

number of training examples = 510
number of test examples = 90
X_train shape: (510, 2500)
Y_train shape: (1, 510)
X_test shape: (90, 2500)
Y_test shape: (1, 90)


In [ ]:

train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[0], train_y.shape[1])
test_y = test_y.reshape(test_y.shape[0], test_y.shape[1])
X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)
print("X_train shape: " + str(X_train.shape))
print("X_test shape: " + str(X_test.shape))

model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=10, batch_size=64,
          callbacks=callbacks_list)
scores = model.evaluate(X_test, test_y, verbose=0)
print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
print_summary(model)

model.save('emojinator.h5')


main()

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.utils import to_categorical


model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Conv2D(64, (5, 5), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(num_of_classes, activation='softmax'))


In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

model = load_model('emojinator.h5')

def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

Using TensorFlow backend.


0
1
2
3
4
5
6
7
8
9
10
11
12
7 0.9983563
4 0.91866404
5 0.6643509
2 0.8742462
5 0.9997991
5 0.9999305
5 0.9998876
5 0.9036536
5 0.6647084
5 0.7747986
5 0.62610155
5 0.9066677
5 0.998223
5 0.9965048
5 0.999902
5 0.9999126
5 0.99995244
5 0.99992895
5 0.9999212
5 0.9994351
5 0.9997359
5 0.9982601
5 0.9994487
5 0.9983467
5 0.9916967
5 0.992878
5 0.97525644
5 0.9818876
5 0.96495825
5 0.96999085
5 0.99171656
5 0.99171656
5 0.99771273
5 0.9971091
5 0.9963084
5 0.9986564
5 0.99845517
5 0.9993262
5 0.9997013
5 0.9993901
5 0.9997067
5 0.99932265
5 0.99168116
5 0.9960633
5 0.9972031
5 0.99806434
5 0.99968195
5 0.99901307
5 0.99978954
5 0.99987495
5 0.9998362
5 0.9998118
5 0.9992329
5 0.99985087
5 0.9999162
5 0.99937576
5 0.9993044
5 0.99987876
5 0.9999293
5 0.9986658
5 0.99992895
5 0.9998379
5 0.9991805
5 0.9991805
5 0.99977165
5 0.9978707
1 0.95740014
5 0.85424083
5 0.9980508
5 0.9998814
5 0.99999964
5 0.9999993
5 0.9999989
5 0.9999993
5 0.9999993
5 0.9999833
5 0.99998736
5 0.9999831
5 0.9999946

2 0.89392453
5 0.9735488
5 0.96936876
5 0.8544537
5 0.7463471
2 0.90457326
2 0.90457326
3 0.4738786
5 0.9796562
5 0.9718346
3 0.743874
5 0.98515517
11 0.35717815
5 0.9890261
3 0.27539274
5 0.8760881
3 0.5450638
5 0.9962535
5 0.9962535
2 0.6299415
2 0.933932
2 0.33009198
2 0.59008527
3 0.5776574
3 0.44069532
1 0.554664
5 0.77473414
1 0.7550504
1 0.99965763
5 0.9145019
1 0.9999347
1 0.9998903
5 0.78836924
1 0.99996793
1 0.8325397
5 0.5648289
1 0.9996648
5 0.6999976
5 0.50594443
1 0.99962044
5 0.86468595
1 0.90448177
1 0.9999882
1 0.9824547
1 0.999471
1 0.9995592
5 0.69025975
5 0.78458405
5 0.6528014
5 0.97529036
1 0.96110183
5 0.7712192
5 0.8926248
1 0.99987686
1 0.9998789
5 0.7757247
5 0.66982645
1 0.9997522
5 0.92702955
1 0.9896651
1 0.7050311
1 0.5429703
5 0.9525518
1 0.99960274
5 0.72375065
1 0.7920697
1 0.9997731
5 0.72074056
1 0.9999888
1 0.99968123
5 0.6446553
1 0.81280696
1 0.99998474
1 0.99985874
1 0.84047925
1 0.55319
1 0.99999547
1 0.9990113
1 0.89513206
1 0.9719981
1 0.957237

5 0.5673497
5 0.9967077
1 0.9861026
1 0.91812456
1 0.9977155
1 0.9133689
3 0.7587658
6 0.92828786
6 0.9928434
4 0.9151412
4 0.7742384
4 0.9657371
4 0.8297996
4 0.7656206
3 0.9998115
3 0.9999255
3 0.82644296
3 0.8557086
3 0.5411353
2 0.8860497
2 0.999826
2 0.9974892
2 0.9999044
2 0.9992668
2 0.9994715
2 0.9994916
2 0.9995933
2 0.9997745
2 0.9997445
2 0.9997918
2 0.9995739
2 0.99966335
2 0.9997861
2 0.9997702
2 0.9998809
2 0.99989545
2 0.9999088
2 0.99949265
2 0.9993117
4 0.41320434
5 0.9971842
5 0.8503231
9 0.9113273
5 0.6044405
5 0.9999584
5 0.99997807
5 0.99998164
5 0.99991715
5 0.9999896
5 0.9999684
5 0.9999895
5 0.9999927
5 0.9999939
5 0.9999819
5 0.9999851
5 0.99998903
5 0.99998856
5 0.99995375
5 0.99998593
5 0.9999943
5 0.99997807
5 0.99997926
5 0.9999924
5 0.99998677
5 0.9999757
5 0.99999225
5 0.99999225
5 0.9999839
5 0.9999627
5 0.99991477
5 0.9999505
5 0.9998116
5 0.9997483
5 0.9991548
5 0.9993112
5 0.9997552
5 0.9994747
5 0.9997713
5 0.9998747
5 0.99988425
5 0.9983016
5 0.9996

5 0.9050421
5 0.9050421
5 0.8937301
5 0.9200444
5 0.6082703
5 0.8198004
5 0.6128166
5 0.7485017
5 0.44304493
5 0.5693127
5 0.53383696
5 0.9708337
5 0.9708337
5 0.79828155
1 0.5367189
5 0.71520364
1 0.46235874
5 0.93383646
5 0.7161349
5 0.7443189
5 0.5975657
1 0.64787245
5 0.4130403
5 0.5096009
1 0.3965729
5 0.7940208
1 0.5311804
5 0.6930257
5 0.89132845
1 0.4895732
5 0.49947485
5 0.5072704
5 0.756801
5 0.9474496
5 0.5564581
5 0.39636087
5 0.88664055
5 0.54039454
5 0.97576195
11 0.83718485
1 0.76238465
9 0.9247469
9 0.7935358
11 0.79056937
11 0.79056937
11 0.99993646
11 0.85427475
1 0.90399843
1 0.97455364
5 0.9998093
5 0.9998807
5 0.99807703
5 0.9999567
5 0.9999621
5 0.99996555
5 0.99996555
5 0.99985147
5 0.98143125
5 0.9951037
5 0.99987423
5 0.77200407
1 0.9252642
5 0.95522064
5 0.9997428
5 0.84421074
5 0.84421074
5 0.99471927
5 0.50250953
1 0.6159359
5 0.99997985
5 0.9999964
5 0.9996387
5 0.99920964
5 0.96216536
1 0.6291188
1 0.97750175
1 0.97750175
1 0.94208866
1 0.8428908
1 0.88007

...

5 0.99999845
5 0.99999976
5 0.99999595
5 0.99999976
5 0.99999976
5 0.99999976
5 0.99999976
5 0.9999988
5 0.9999988
5 0.99999845
5 0.99999607
5 0.99352926
5 0.9920676
5 0.994263
5 0.90908134
5 0.91178083
1 0.99417734
1 0.9289601
1 0.9922007
1 0.9922007
1 0.9753711
1 0.5424305
5 0.80972844
5 0.9973241
5 0.99450463
1 0.9838675
1 0.9227729
1 0.96040475
1 0.6820086
1 0.9771549
1 0.9168648
5 0.99318707
1 0.6177964
5 0.9748377
5 0.8448323
1 0.9517883
1 0.8382958
5 0.7437874
1 0.91857123
1 0.69158167
5 0.8806819
1 0.92294234
1 0.55702573
1 0.740764
1 0.9477924
5 0.9749161
1 0.85751283
5 0.80509555
1 0.8088228
5 0.9993318
5 0.86245155
5 0.8528548
5 0.70315945
5 0.72255105
1 0.86541307
1 0.8712117
1 0.87802625
2 0.9985512
1 0.9235735
1 0.94218993
2 0.50714636
11 0.9744553
11 0.9744553
11 0.8391394
4 0.9937708
4 0.8656663
6 0.5861237
4 0.90951425
11 0.810044
11 0.98374707
11 0.6260907
1 0.82528776
1 0.59749097
1 0.9631811
1 0.8722796
1 0.7011944
1 0.93444407
1 0.77381235
1 0.8128269
1 0.7457704
1

4 0.37689915
4 0.47696057
6 0.6568512
4 0.8884741
4 0.6273706
4 0.9502671
4 0.7136046
4 0.35393655
4 0.63112897
4 0.91463596
4 0.73831445
4 0.6589333
6 0.6010388
6 0.6776112
6 0.5193315
4 0.56927335
4 0.2812959
6 0.63918203
6 0.70669544
4 0.87851524
4 0.5585301
4 0.66551393
4 0.3014986
4 0.3014986
4 0.90590984
4 0.90877
4 0.9893845
4 0.77190554
4 0.7680645
4 0.9323936
4 0.76930887
4 0.66544443
4 0.9433586
4 0.8958526
4 0.8958526
4 0.9218447
4 0.7028195
4 0.88458604
4 0.9513399
4 0.93173265
4 0.70821047
4 0.92969733
4 0.8875353
4 0.8995879
4 0.83863896
4 0.89414674
4 0.89414674
4 0.96593493
4 0.790636
6 0.40857595
4 0.7487016
4 0.33977985
4 0.97385323
4 0.6735099
4 0.7448116
4 0.45904362
4 0.9242244
4 0.9242244
4 0.84920347
4 0.8481762
6 0.34852612
4 0.96591914
4 0.3987592
6 0.5996648
4 0.7398834
4 0.7728737
4 0.82084644
4 0.37685066
4 0.37685066
4 0.44388545
6 0.60406655
6 0.7390909
6 0.58084774
6 0.43537647
4 0.59006435
6 0.6695686
6 0.5405127
4 0.5647307
4 0.83590305
4 0.83590305
4 0

2 0.59972775
2 0.9982792
2 0.9990208
2 0.9963707
2 0.99718755
2 0.9990338
2 0.99423885
2 0.9922131
2 0.99821836
2 0.9491831
2 0.9994043
2 0.9971149
2 0.98581576
2 0.99897707
2 0.99733365
2 0.99433583
2 0.9966216
1 0.8430344
5 0.6734862
5 0.9998416
5 0.9999981
5 0.9999999
5 0.99999976
5 0.9999559
5 0.9999163
5 0.99998534
5 0.99997926
5 0.9999939
5 0.9999976
5 0.99999356
5 0.99999964
5 0.9999987
5 0.99997187
5 0.9999963
5 0.99994767
5 0.99999034
5 0.99999726
5 0.99999285
5 0.99999523
5 0.9999995
5 0.96854717
1 0.9746166
2 0.9887545
2 0.986257
2 0.7533614
2 0.97158426
3 0.656214
2 0.6298618
2 0.89403427
2 0.9852573
2 0.8252234
2 0.66236025
2 0.9858457
3 0.5319546
2 0.97877353
2 0.9974304
3 0.7958719
2 0.8961518
2 0.9794654
3 0.5332001
2 0.99843305
2 0.97415584
3 0.77295595
2 0.9960162
2 0.5870549
2 0.77359426
2 0.9913017
3 0.7010464
3 0.52792346
2 0.9806768
2 0.9906759
2 0.9414078
2 0.7399655
2 0.9760296
2 0.982156
2 0.9976433
2 0.9324997
2 0.94045204
2 0.9996636
2 0.9409507
2 0.71680003


1 0.6963114
5 0.69472575
5 0.7930165
5 0.7565399
1 0.88248414
1 0.9591454
1 0.99997544
1 0.46967965
3 0.9563916
3 0.9671441
3 0.9671441
8 0.81303525
8 0.9912211
8 0.9379207
11 0.9579988
11 0.9971553
11 0.9980896
11 0.9999405
11 0.99997616
11 0.9996933
11 0.9999728
11 0.9999728
11 0.99996686
11 0.9999119
11 0.9997135
11 0.9983181
11 0.99936324
11 0.9996006
11 0.9975043
11 0.9986136
11 0.9948526
11 0.97199684
11 0.97199684
1 0.8113309
3 0.77317536
6 0.5803233
3 0.45776886
3 0.89244264
3 0.3254989
3 0.8639787
3 0.7172331
3 0.37299052
1 0.79453117
3 0.7182889
3 0.7182889
8 0.66310287
8 0.87648827
1 0.9999988
1 0.99999857
1 0.99999964
1 0.99999726
1 0.9999876
1 0.9999974
1 0.9999932
1 0.9999958
1 0.9999958
1 0.9999976
1 0.99999607
1 0.9999957
1 0.9999969
1 0.9999982
1 0.99999666
1 0.9999943
1 0.9999707
1 0.6555924
8 0.61391056
4 0.99480987
4 0.99480987
11 0.95134425
2 0.6498965
2 0.80610836
4 0.57453835
4 0.9555871
4 0.9972397
4 0.9480803
4 0.93571466
3 0.7050483
5 0.9402588
5 0.9402588
5 0

KeyboardInterrupt: 